In [1]:
import pandas as pd
import numpy as np
import re
from pdfminer.high_level import extract_text
from unidecode import unidecode
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Load the data
df = pd.read_json("../Data/hearings_witnesses_contrarians_classified_categorised.json")
# Append the MoCs to the data
mocs = pd.read_csv("../Data/hearings_committee_member_level.csv")
df["hearing_id"] = df["packageId"].str[5:]
df = df.merge(pd.DataFrame(mocs.groupby("hearing_id").last_name.apply(list).reset_index(name='moc_name')), on = "hearing_id")
df = df.merge(pd.DataFrame(mocs.groupby("hearing_id").bioguide_id.apply(list).reset_index(name='bioguide_id')), on = "hearing_id")
df = df.merge(pd.DataFrame(mocs.groupby("hearing_id").govtrack_id.apply(list).reset_index(name='govtrack_id')), on = "hearing_id")
df.keys()

Index(['packageId', 'lastModified', 'packageLink', 'docClass', 'title',
       'congress', 'dateIssued', 'htm', 'mods', 'htm_stripped',
       'keywords_climatechange', 'keywords_climatefocus',
       'keywords_capandtrade', 'identifier', 'chamber', 'committee',
       'committee_short', 'witnesses', 'contrarian_actor',
       'contrarian_organisation', 'contrarian', 'denialist',
       'witness_affiliation', 'witness_sector', 'witness_industry',
       'state_research', 'subcategory', 'category', 'hearing_id', 'moc_name',
       'bioguide_id', 'govtrack_id'],
      dtype='object')

In [3]:
# Add second committee to the multi-committee hearing 110hhrg37579
df.committee[34] = [df.committee[34], mocs.committee_short[300]]
df.moc_name[34] = df.moc_name[34] + mocs[mocs.hearing_id == "110hhrg32966"].groupby("hearing_id").last_name.apply(list).reset_index(name='moc_name').moc_name[0]
df.bioguide_id[34] = df.bioguide_id[34] + mocs[mocs.hearing_id == "110hhrg32966"].groupby("hearing_id").bioguide_id.apply(list).reset_index(name='bioguide_id').bioguide_id[0]
df.govtrack_id[34] = df.govtrack_id[34] + mocs[mocs.hearing_id == "110hhrg32966"].groupby("hearing_id").govtrack_id.apply(list).reset_index(name='govtrack_id').govtrack_id[0]

In [4]:
# Address data issues

# df.htm[17] # Hearing CHRG-110hhrg32966
# # Three day hearing, however, only day one has been transcribed into the htm. Day two and three only available in pdf form.
# https://www.congress.gov/110/chrg/CHRG-110hhrg32966/CHRG-110hhrg32966.pdf
# https://www.govinfo.gov/app/details/CHRG-110hhrg32966/CHRG-110hhrg32966/summary
# https://science.house.gov/hearings?ID=9972A59C-1C1C-45A2-8D05-8E399BBCBF02
# https://science.house.gov/hearings?ID=68F05FE7-DD6E-440A-A7D7-4AB39B4C380B

# Replace htm with pdf text

# Parse the pdf
text_raw = extract_text("../Data/hearings/CHRG-110hhrg32966.pdf")

panel1 = re.compile("(?:\x0c7\n)(.+?)(?:\[Whereupon)", re.DOTALL)
panel2 = re.compile("(?:\x0c119\n)(.+?)(?:\[Whereupon)", re.DOTALL)
panel3 = re.compile("(?:\x0c237\n)(.+?)(?:\[Whereupon)", re.DOTALL)

text = "    ".join([re.findall(panel1, text_raw)[0], re.findall(panel2, text_raw)[0], re.findall(panel3, text_raw)[0]])

# Tidy the parsed text
text = text.replace("-\n", "")
regex = re.compile("\n*VerDate.+?\n\n\x0c\d+\n\n(?!Ms\.|Mrs\.|Mr\.|Dr\.|Chairman|Senator|Secretary|Administrator|Governor|Mayor|Ambassador|Sir|Lord|Von|Bishop|Reverend|Rev\.|Rabbi|Admiral|General|Lieutenant)", re.DOTALL)
text = (re.sub(regex, " ", text))
regex = re.compile("-\n*VerDate.+?\n\n\x0c\d+\n\n", re.DOTALL)
text = (re.sub(regex, "", text))
regex = re.compile("\n*VerDate.+?\n\n\x0c\d+\n\n", re.DOTALL)
text = (re.sub(regex, "\n", text))

text = text.replace("  ", " ")
text = re.sub("(?<=[a-z])\n(?=[a-z])", " ", text)
text = re.sub("(?<=\.)\n", "    \n", text)
text = re.sub("((?<!    )\n\n)", " ", text)
text = re.sub("((?<!    )\n(?!\n))", " ", text)
text = re.sub("\n", "\n    ", text)
text = re.sub("\n     ", "\n    ", text)

regex_split = "((?<=\S) (?=(Ms\.|Mrs\.|Mr\.|Dr\.|Chairman|Senator|Secretary|Administrator|Governor|Mayor|Ambassador|Sir|Lord|Von|Bishop|Reverend|Rev\.|Rabbi|Admiral|General|Lieutenant) ([A-Z]{3,})\.))"
text = re.sub(regex_split, "\n    ", text)
text = "\n".join([sentence for sentence in text.split("\n") if not sentence.isupper()])
text = re.sub(" Chairman GORDON. Well, good morning,", "    Chairman GORDON. Well, good morning,", text)

df.loc[17, 'htm'] = text

In [5]:
df.head(2)

,packageId,lastModified,packageLink,docClass,title,congress,dateIssued,htm,mods,htm_stripped,...,witness_affiliation,witness_sector,witness_industry,state_research,subcategory,category,hearing_id,moc_name,bioguide_id,govtrack_id
0,CHRG-108shrg82493,2022-01-13T03:11:07Z,https://api.govinfo.gov/packages/CHRG-108shrg8...,SHRG,The Impacts of Climate Change and States' Actions,108,2004-05-06,<html>\n<title> - S. Hrg. 108-966 THE IMPACTS ...,"<mods xmlns:xlink=""http://www.w3.org/1999/xlin...",<html> <title> - s. hrg. 108-966 the impacts o...,...,[Northeast States for Coordinated Air Use Mana...,"[Other, Other, Other, Ideological/Single-Issue...","[Civil Servants/Public Officials, Non-Profit I...","[0, 0, 0, 0, 0]","[Government Officials, Nonpartisan Non-Profits...","[Government Officials, Non-Profit Organisation...",108shrg82493,"[McCain, Stevens, Burns, Lott, Hutchison, Snow...","[M000303, S000888, B001126, L000447, H001016, ...","[300071, 300094, 300015, 300069, 300054, 30009..."
1,CHRG-108hhrg88427,2022-01-13T02:35:39Z,https://api.govinfo.gov/packages/CHRG-108hhrg8...,HHRG,The Clear Skies Initiative: A Multipollutant A...,108,2003-07-08,<html>\n<title> - THE CLEAR SKIES INITIATIVE: ...,"<mods xmlns:xlink=""http://www.w3.org/1999/xlin...",<html> <title> - the clear skies initiative: a...,...,[Environmental Protection Agency],[Other],[Civil Servants/Public Officials],[0],[Government Officials],[Government Officials],108hhrg88427,"[Tauzin, Bilirakis, Barton, Upton, Stearns, Gi...","[T000058, B000463, B000213, U000031, S000822, ...","[400397, 400028, 400018, 400414, 400388, 40015..."


# Prepare the witness and MoC names for matching

In [6]:
# Extract and correct last names

## Extract witness last names
df['witness_name'] = df.witnesses.apply(
    lambda x: [i.split(',')[0] for i in x]).apply(
    lambda x: [re.sub(r'((Ms|Mrs|Mr|Dr)(\.|\s)){1}', '', i) for i in x]).apply(
    lambda x: [re.sub(r'([A-Z]\.){1,2}', '', i).strip() for i in x]).apply(
    lambda x: [i.split()[-1] if len(i.split())>1 else i for i in x])

# Correct misspelled names
df.witness_name[3][16] = "Kroszner"
df.witness_name[10][1] = "Lawson"
df.witness_name[12][5] = "Von Storch"
df.witness_name[30][8] = "Sharkey"
df.witness_name[42][0] = "Stern"
df.witness_name[43][1] = "Holliday"
df.witness_name[45][0] = "Gabriel"
df.witness_name[71][2] = "Dellasala"
df.witness_name[74][1] = "Sullivan"
df.moc_name[34][47] = "Bono"

# Adapt duplicate family names to match the transcripts
df.moc_name[23][35] = "Smith of Nebraska"
df.moc_name[67][51] = "Rogers of Michigan"
df.moc_name[21][8] = "Johnson of Texas"
df.witness_name[91][36] = "Mr. Smith"
df.witness_name[91][66] = "Ms. Smith"

# Adapt hearings with divergent new line pattern
df.htm[12] = df.htm[12].replace("\t", "    ")

# Adapt hearings with divergent or varying adress patterns
df.htm[12] = re.sub("\n(?=MR|MRS|DR)", "\n    ", df.htm[12])
df.htm[61] = re.sub("\s{4}Mr\. Gore\s{2}", "    Mr. Gore.", df.htm[61])
df.htm[61] = re.sub("\s{4}Senator\. (?=\w{3,}\.\s)", "    Senator ", df.htm[61])
df.htm[75] = re.sub("Mr\. Golden\.", "The Chairman.", df.htm[75])
df.htm[83] = re.sub("\[Mr\. Chu\'s response follows\:\]", "Mr. Chu.", df.htm[83])

# Correct misspelled witness name in text
df.htm[3] = df.htm[3].replace("Napper", "Nappier")
df.htm[12] = df.htm[12].replace("   Dr. Roger Ravelle", "Dr. Roger Ravelle")
df.htm[12] = df.htm[12].replace("        MRS. BLACKBURN.", "    MRS. BLACKBURN.")
df.htm[12] = df.htm[12].replace("MAT \n    Lab", "MAT Lab")
df.htm[20] = df.htm[20].replace("Mullaly", "Mulally")
df.htm[37] = df.htm[37].replace("NICOLS", "Nichols")
df.htm[61] = df.htm[61].replace("Alaxander", "Alexander")
df.htm[61] = df.htm[61].replace("Milkulski", "Mikulski")
df.htm[78] = df.htm[78].replace("Lujan", "Luján")
df.htm[86] = df.htm[86].replace("Welsh", "Welch")
df.htm[89] = df.htm[89].replace("Velazquez", "Velázquez")
df.htm[89] = df.htm[89].replace("Leutkemeyer", "Luetkemeyer")
df.htm[90] = df.htm[90].replace("Corkecr", "Corker")
df.htm[90] = df.htm[90].replace("The prepared statement of Mr. Helme follows:", "[The prepared statement of Mr. Helme follows:]")
df.htm[96] = df.htm[96].replace("SANCHEZ", "SÁNCHEZ")
df.htm[112] = df.htm[112].replace("Lujan", "Luján")
df.htm[112] = df.htm[112].replace("Dalhkemper", "Dahlkemper")

In [7]:
## Remove duplicate witnesses
# Morris, Michael
df.witness_name[6] = [i for j, i in enumerate(df.witness_name[6]) if not j == 26]
df.witnesses[6] = [i for j, i in enumerate(df.witnesses[6]) if not j == 26]
df.contrarian_actor[6] = [i for j, i in enumerate(df.contrarian_actor[6]) if not j == 26]
df.contrarian_organisation[6] = [i for j, i in enumerate(df.contrarian_organisation[6]) if not j == 26]
df.contrarian[6] = [i for j, i in enumerate(df.contrarian[6]) if not j == 26]
df.denialist[6] = [i for j, i in enumerate(df.denialist[6]) if not j == 26]
df.witness_affiliation[6] = [i for j, i in enumerate(df.witness_affiliation[6]) if not j == 26]
df.witness_sector[6] = [i for j, i in enumerate(df.witness_sector[6]) if not j == 26]
df.witness_industry[6] = [i for j, i in enumerate(df.witness_industry[6]) if not j == 26]
df.state_research[6] = [i for j, i in enumerate(df.state_research[6]) if not j == 26]
df.subcategory[6] = [i for j, i in enumerate(df.subcategory[6]) if not j == 26]
df.category[6] = [i for j, i in enumerate(df.category[6]) if not j == 26]
# Hayes, David J.
df.witness_name[22] = [i for j, i in enumerate(df.witness_name[22]) if not j == 3]
df.witnesses[22] = [i for j, i in enumerate(df.witnesses[22]) if not j == 3]
df.contrarian_actor[22] = [i for j, i in enumerate(df.contrarian_actor[22]) if not j == 3]
df.contrarian_organisation[22] = [i for j, i in enumerate(df.contrarian_organisation[22]) if not j == 3]
df.contrarian[22] = [i for j, i in enumerate(df.contrarian[22]) if not j == 3]
df.denialist[22] = [i for j, i in enumerate(df.denialist[22]) if not j == 3]
df.witness_affiliation[22] = [i for j, i in enumerate(df.witness_affiliation[22]) if not j == 3]
df.witness_sector[22] = [i for j, i in enumerate(df.witness_sector[22]) if not j == 3]
df.witness_industry[22] = [i for j, i in enumerate(df.witness_industry[22]) if not j == 3]
df.state_research[22] = [i for j, i in enumerate(df.state_research[22]) if not j == 3]
df.subcategory[22] = [i for j, i in enumerate(df.subcategory[22]) if not j == 3]
df.category[22] = [i for j, i in enumerate(df.category[22]) if not j == 3]

In [8]:
## Add missing witness
# Holmstead, Hon. Jeffrey (accompanying Ms. Christine Todd Whitman)
df.witness_name[3].append(df.witness_name[1][0])
df.witnesses[3].append(df.witnesses[1][0])
df.contrarian_actor[3].append(df.contrarian_actor[1][0])
df.contrarian_organisation[3].append(df.contrarian_organisation[1][0])
df.contrarian[3].append(df.contrarian[1][0])
df.denialist[3].append(df.denialist[1][0])
df.witness_affiliation[3].append(df.witness_affiliation[1][0])
df.witness_sector[3].append(df.witness_sector[1][0])
df.witness_industry[3].append(df.witness_industry[1][0])
df.state_research[3].append(df.state_research[1][0])
df.subcategory[3].append(df.subcategory[1][0])
df.category[3].append(df.category[1][0])
# Hewson, Tom (accompanying Mr. Joel Bluestein)
df.witness_name[3].append("Hewson")
df.witnesses[3].append("Hewson, Tom")
df.contrarian_actor[3].append(df.contrarian_actor[3][7])
df.contrarian_organisation[3].append(df.contrarian_organisation[3][7])
df.contrarian[3].append(df.contrarian[3][7])
df.denialist[3].append(df.denialist[3][7])
df.witness_affiliation[3].append(df.witness_affiliation[3][7])
df.witness_sector[3].append(df.witness_sector[3][7])
df.witness_industry[3].append(df.witness_industry[3][7])
df.state_research[3].append(df.state_research[3][7])
df.subcategory[3].append(df.subcategory[3][7])
df.category[3].append(df.category[3][7])
# Lindstrom, Perry (called to explain a prepared chart)
df.witness_name[6].append("Lindstrom")
df.witnesses[6].append("Lindstrom, Perry")
df.contrarian_actor[6].append(df.contrarian_actor[62][3])
df.contrarian_organisation[6].append(df.contrarian_organisation[62][3])
df.contrarian[6].append(df.contrarian[62][3])
df.denialist[6].append(df.denialist[62][3])
df.witness_affiliation[6].append(df.witness_affiliation[62][3])
df.witness_sector[6].append(df.witness_sector[62][3])
df.witness_industry[6].append(df.witness_industry[62][3])
df.state_research[6].append(df.state_research[62][3])
df.subcategory[6].append(df.subcategory[62][3])
df.category[6].append(df.category[62][3])
# Dr. Montgomery
df.witness_name[9].append("Montgomery")
df.witnesses[9].append("Dr. Montgomery of Charles River Associates, CRA International")
df.contrarian_actor[9].append(df.contrarian_actor[9][6])
df.contrarian_organisation[9].append(df.contrarian_organisation[9][6])
df.contrarian[9].append(df.contrarian[9][6])
df.denialist[9].append(df.denialist[9][6])
df.witness_affiliation[9].append(df.witness_affiliation[9][6])
df.witness_sector[9].append(df.witness_sector[9][6])
df.witness_industry[9].append(df.witness_industry[9][6])
df.state_research[9].append(df.state_research[9][6])
df.subcategory[9].append(df.subcategory[9][6])
df.category[9].append(df.category[9][6])
# Bloomfield, Peter, accompanying Dr. North
df.witness_name[12].append("Bloomfield")
df.witnesses[12].append("Peter Bloomfield, professor of statistics, North Carolina State University")
df.contrarian_actor[12].append(df.contrarian_actor[12][1])
df.contrarian_organisation[12].append(df.contrarian_organisation[12][1])
df.contrarian[12].append(df.contrarian[12][1])
df.denialist[12].append(df.denialist[12][1])
df.witness_affiliation[12].append(df.witness_affiliation[12][1])
df.witness_sector[12].append(df.witness_sector[12][1])
df.witness_industry[12].append(df.witness_industry[12][1])
df.state_research[12].append(df.state_research[12][1])
df.subcategory[12].append(df.subcategory[12][1])
df.category[12].append(df.category[12][1])
# Shea, Quin
df.witness_name[14].append("Shea")
df.witnesses[14].append("Quin Shea, Senior Director for Environmental Activities, Edison Electric Institute")
df.contrarian_actor[14].append(df.contrarian_actor[3][4])
df.contrarian_organisation[14].append(df.contrarian_organisation[3][4])
df.contrarian[14].append(df.contrarian[3][4])
df.denialist[14].append(df.denialist[3][4])
df.witness_affiliation[14].append(df.witness_affiliation[3][4])
df.witness_sector[14].append(df.witness_sector[3][4])
df.witness_industry[14].append(df.witness_industry[3][4])
df.state_research[14].append(df.state_research[3][4])
df.subcategory[14].append(df.subcategory[3][4])
df.category[14].append(df.category[3][4])
# Solomon, Susan
df.witness_name[17].append("Solomon")
df.witnesses[17].append("Dr. Susan Solomon, Co-Chair, IPCC, Working Group I: The Physical Basis of Climate Change; Senior Scientist, Earth System Research Laboratory, Office of Oceanic and Atmospheric Research, National Oceanic and Atmospheric Administration, U.S")
df.contrarian_actor[17].append(df.contrarian_actor[40][2])
df.contrarian_organisation[17].append(df.contrarian_organisation[40][2])
df.contrarian[17].append(df.contrarian[40][2])
df.denialist[17].append(df.denialist[40][2])
df.witness_affiliation[17].append(df.witness_affiliation[40][2])
df.witness_sector[17].append(df.witness_sector[40][2])
df.witness_industry[17].append(df.witness_industry[40][2])
df.state_research[17].append(df.state_research[40][2])
df.subcategory[17].append(df.subcategory[40][2])
df.category[17].append(df.category[40][2])
# Trenberth, Kevin E. 
df.witness_name[17].append("Trenberth")
df.witnesses[17].append("Dr. Kevin E. Trenberth, Coordinating Lead Author, IPCC, Working Group I, Chapter 3: Observations: Surface and Atmospheric Climate Change; Head, Climate Analysis Section, National Center for Atmospheric Research")
df.contrarian_actor[17].append(df.contrarian_actor[40][2])
df.contrarian_organisation[17].append(df.contrarian_organisation[40][2])
df.contrarian[17].append(df.contrarian[40][2])
df.denialist[17].append(df.denialist[40][2])
df.witness_affiliation[17].append(df.witness_affiliation[40][2])
df.witness_sector[17].append(df.witness_sector[40][2])
df.witness_industry[17].append(df.witness_industry[40][2])
df.state_research[17].append(df.state_research[40][2])
df.subcategory[17].append(df.subcategory[40][2])
df.category[17].append(df.category[40][2])
# Alley, Richard B.
df.witness_name[17].append("Alley")
df.witnesses[17].append("Dr. Richard B. Alley, Lead Author, IPCC, Working Group I, Chapter 4: Observations: Changes in Snow, Ice and Frozen Ground; Evan Pugh Professor of Geosciences and Associate of the Earth and Environmental Systems Institute, Pennsylvania State University")
df.contrarian_actor[17].append(df.contrarian_actor[40][2])
df.contrarian_organisation[17].append(df.contrarian_organisation[40][2])
df.contrarian[17].append(df.contrarian[40][2])
df.denialist[17].append(df.denialist[40][2])
df.witness_affiliation[17].append(df.witness_affiliation[40][2])
df.witness_sector[17].append(df.witness_sector[40][2])
df.witness_industry[17].append(df.witness_industry[40][2])
df.state_research[17].append(df.state_research[40][2])
df.subcategory[17].append(df.subcategory[40][2])
df.category[17].append(df.category[40][2])
# Meehl, Gerald A.
df.witness_name[17].append("Meehl")
df.witnesses[17].append("Dr. Gerald A. Meehl, Coordinating Lead Author, IPCC, Working Group I, Chapter 10: Global Climate Projections; Senior Scientist, National Center for Atmospheric Research")
df.contrarian_actor[17].append(df.contrarian_actor[40][2])
df.contrarian_organisation[17].append(df.contrarian_organisation[40][2])
df.contrarian[17].append(df.contrarian[40][2])
df.denialist[17].append(df.denialist[40][2])
df.witness_affiliation[17].append(df.witness_affiliation[40][2])
df.witness_sector[17].append(df.witness_sector[40][2])
df.witness_industry[17].append(df.witness_industry[40][2])
df.state_research[17].append(df.state_research[40][2])
df.subcategory[17].append(df.subcategory[40][2])
df.category[17].append(df.category[40][2])
# Galloway, Jerry
df.witness_name[21].append("Galloway")
df.witnesses[21].append("Jerry Galloway, President of the American Water Resources Association")
df.contrarian_actor[21].append(df.contrarian_actor[0][1])
df.contrarian_organisation[21].append(df.contrarian_organisation[0][1])
df.contrarian[21].append(df.contrarian[0][1])
df.denialist[21].append(df.denialist[0][1])
df.witness_affiliation[21].append(df.witness_affiliation[0][1])
df.witness_sector[21].append(df.witness_sector[0][1])
df.witness_industry[21].append(df.witness_industry[0][1])
df.state_research[21].append(df.state_research[0][1])
df.subcategory[21].append(df.subcategory[0][1])
df.category[21].append(df.category[0][1])
# Peters, Siobhan, accompanying Stern, Sir Nicholas
df.witness_name[27].append("Peters")
df.witnesses[27].append("Siobhan Peters, head of the review team of the Stern Review")
df.contrarian_actor[27].append(df.contrarian_actor[27][0])
df.contrarian_organisation[27].append(df.contrarian_organisation[27][0])
df.contrarian[27].append(df.contrarian[27][0])
df.denialist[27].append(df.denialist[27][0])
df.witness_affiliation[27].append(df.witness_affiliation[27][0])
df.witness_sector[27].append(df.witness_sector[27][0])
df.witness_industry[27].append(df.witness_industry[27][0])
df.state_research[27].append(df.state_research[27][0])
df.subcategory[27].append(df.subcategory[27][0])
df.category[27].append(df.category[27][0])
# Kelly, Glenn, accompanying Kraig R. Naasz
df.witness_name[38].append("Kelly")
df.witnesses[38].append("Glenn Kelly, Vice President, Government Affairs, National Mining Association")
df.contrarian_actor[38].append(df.contrarian_actor[38][0])
df.contrarian_organisation[38].append(df.contrarian_organisation[38][0])
df.contrarian[38].append(df.contrarian[38][0])
df.denialist[38].append(df.denialist[38][0])
df.witness_affiliation[38].append(df.witness_affiliation[38][0])
df.witness_sector[38].append(df.witness_sector[38][0])
df.witness_industry[38].append(df.witness_industry[38][0])
df.state_research[38].append(df.state_research[38][0])
df.subcategory[38].append(df.subcategory[38][0])
df.category[38].append(df.category[38][0])
# Reifsnyder, Dan, accompanying Dobriansky, Hon. Paula J.
df.witness_name[52].append("Reifsnyder")
df.witnesses[52].append("Dan Reifsnyder, Deputy Assistant Secretary, Bureau of Oceans Environment Science, Department of State, Washington, DC")
df.contrarian_actor[52].append(df.contrarian_actor[52][0])
df.contrarian_organisation[52].append(df.contrarian_organisation[52][0])
df.contrarian[52].append(df.contrarian[52][0])
df.denialist[52].append(df.denialist[52][0])
df.witness_affiliation[52].append(df.witness_affiliation[52][0])
df.witness_sector[52].append(df.witness_sector[52][0])
df.witness_industry[52].append(df.witness_industry[52][0])
df.state_research[52].append(df.state_research[52][0])
df.subcategory[52].append(df.subcategory[52][0])
df.category[52].append(df.category[52][0])
# CHRG-111hhrg63514 We are going to pick up questions where we left off, but I want to introduce a new set of witnesses.
# Mr. Pershing.
df.witness_name[67].append("Pershing")
df.witnesses[67].append("Mr. Pershing.  Jonathan Pershing with the World Resources Institute")
df.contrarian_actor[67].append(df.contrarian_actor[67][8])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][8])
df.contrarian[67].append(df.contrarian[67][8])
df.denialist[67].append(df.denialist[67][8])
df.witness_affiliation[67].append(df.witness_affiliation[67][8])
df.witness_sector[67].append(df.witness_sector[67][8])
df.witness_industry[67].append(df.witness_industry[67][8])
df.state_research[67].append(df.state_research[67][8])
df.subcategory[67].append(df.subcategory[67][8])
df.category[67].append(df.category[67][8])
# Mr. Kline.
df.witness_name[67].append("Kline")
df.witnesses[67].append("Steve Kline, Vice President of Corporate Environmental and Federal Affairs for PG&E Corporation")
df.contrarian_actor[67].append(df.contrarian_actor[67][5])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][5])
df.contrarian[67].append(df.contrarian[67][5])
df.denialist[67].append(df.denialist[67][5])
df.witness_affiliation[67].append(df.witness_affiliation[67][5])
df.witness_sector[67].append(df.witness_sector[67][5])
df.witness_industry[67].append(df.witness_industry[67][5])
df.state_research[67].append(df.state_research[67][5])
df.subcategory[67].append(df.subcategory[67][5])
df.category[67].append(df.category[67][5])
# Ms. Moler.
df.witness_name[67].append("Moler")
df.witnesses[67].append("Betsy Moler, Executive Vice President of Government and Environment Affairs for Exelon Corporation")
df.contrarian_actor[67].append(df.contrarian_actor[67][10])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][10])
df.contrarian[67].append(df.contrarian[67][10])
df.denialist[67].append(df.denialist[67][10])
df.witness_affiliation[67].append(df.witness_affiliation[67][10])
df.witness_sector[67].append(df.witness_sector[67][10])
df.witness_industry[67].append(df.witness_industry[67][10])
df.state_research[67].append(df.state_research[67][10])
df.subcategory[67].append(df.subcategory[67][10])
df.category[67].append(df.category[67][10])
# Ms. Klee.
df.witness_name[67].append("Klee")
df.witnesses[67].append("Ann Renee Klee, Vice President for Corporate Environmental Programs for General Electric")
df.contrarian_actor[67].append(df.contrarian_actor[67][0])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][0])
df.contrarian[67].append(df.contrarian[67][0])
df.denialist[67].append(df.denialist[67][0])
df.witness_affiliation[67].append(df.witness_affiliation[67][0])
df.witness_sector[67].append(df.witness_sector[67][0])
df.witness_industry[67].append(df.witness_industry[67][0])
df.state_research[67].append(df.state_research[67][0])
df.subcategory[67].append(df.subcategory[67][0])
df.category[67].append(df.category[67][0])
# Mr. Corneli.
df.witness_name[67].append("Corneli")
df.witnesses[67].append("Steven B. Corneli, Senior Vice President, Market and Climate Policy for NRG Energy")
df.contrarian_actor[67].append(df.contrarian_actor[67][11])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][11])
df.contrarian[67].append(df.contrarian[67][11])
df.denialist[67].append(df.denialist[67][11])
df.witness_affiliation[67].append(df.witness_affiliation[67][11])
df.witness_sector[67].append(df.witness_sector[67][11])
df.witness_industry[67].append(df.witness_industry[67][11])
df.state_research[67].append(df.state_research[67][11])
df.subcategory[67].append(df.subcategory[67][11])
df.category[67].append(df.category[67][11])
# Ms. Peace.
df.witness_name[67].append("Peace")
df.witnesses[67].append("Janet Peace, Vice President for Markets and Business Strategy for the Pew Center on Global Climate Change")
df.contrarian_actor[67].append(df.contrarian_actor[67][4])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][4])
df.contrarian[67].append(df.contrarian[67][4])
df.denialist[67].append(df.denialist[67][4])
df.witness_affiliation[67].append(df.witness_affiliation[67][4])
df.witness_sector[67].append(df.witness_sector[67][4])
df.witness_industry[67].append(df.witness_industry[67][4])
df.state_research[67].append(df.state_research[67][4])
df.subcategory[67].append(df.subcategory[67][4])
df.category[67].append(df.category[67][4])
# Ms. Thompson.
df.witness_name[67].append("Thompson")
df.witnesses[67].append("Elizabeth Thompson, the Legislative Director of the Environmental Defense Fund")
df.contrarian_actor[67].append(df.contrarian_actor[67][3])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][3])
df.contrarian[67].append(df.contrarian[67][3])
df.denialist[67].append(df.denialist[67][3])
df.witness_affiliation[67].append(df.witness_affiliation[67][3])
df.witness_sector[67].append(df.witness_sector[67][3])
df.witness_industry[67].append(df.witness_industry[67][3])
df.state_research[67].append(df.state_research[67][3])
df.subcategory[67].append(df.subcategory[67][3])
df.category[67].append(df.category[67][3])
# Mr. Bendick.
df.witness_name[67].append("Bendick")
df.witnesses[67].append("Robert L. Bendick, Jr., Director of U.S. Government Relations for The Nature Conservancy")
df.contrarian_actor[67].append(df.contrarian_actor[67][12])
df.contrarian_organisation[67].append(df.contrarian_organisation[67][12])
df.contrarian[67].append(df.contrarian[67][12])
df.denialist[67].append(df.denialist[67][12])
df.witness_affiliation[67].append(df.witness_affiliation[67][12])
df.witness_sector[67].append(df.witness_sector[67][12])
df.witness_industry[67].append(df.witness_industry[67][12])
df.state_research[67].append(df.state_research[67][12])
df.subcategory[67].append(df.subcategory[67][12])
df.category[67].append(df.category[67][12])
# Mr. Kostyack
df.witness_name[72].append("Kostyack")
df.witnesses[72].append("Mr. Kostyack, Vice President, National Wildlife Federation (on behalf on Mr Schweiger)")
df.contrarian_actor[72].append(df.contrarian_actor[72][2])
df.contrarian_organisation[72].append(df.contrarian_organisation[72][2])
df.contrarian[72].append(df.contrarian[72][2])
df.denialist[72].append(df.denialist[72][2])
df.witness_affiliation[72].append(df.witness_affiliation[72][2])
df.witness_sector[72].append(df.witness_sector[72][2])
df.witness_industry[72].append(df.witness_industry[72][2])
df.state_research[72].append(df.state_research[72][2])
df.subcategory[72].append(df.subcategory[72][2])
df.category[72].append(df.category[72][2])
# Mr. Fawcett.
df.witness_name[92].append("Fawcett")
df.witnesses[92].append("Allen Fawcett, accompaying Mr. Harvey, Office of Air and Radiation, Environmental Protection Agency")
df.contrarian_actor[92].append(df.contrarian_actor[92][0])
df.contrarian_organisation[92].append(df.contrarian_organisation[92][0])
df.contrarian[92].append(df.contrarian[92][0])
df.denialist[92].append(df.denialist[92][0])
df.witness_affiliation[92].append(df.witness_affiliation[92][0])
df.witness_sector[92].append(df.witness_sector[92][0])
df.witness_industry[92].append(df.witness_industry[92][0])
df.state_research[92].append(df.state_research[92][0])
df.subcategory[92].append(df.subcategory[92][0])
df.category[92].append(df.category[92][0])

# Parse the utterances

In [9]:
regex_speaker = ("\s{4}(ms\.|mrs\.|mr\.|dr\.|chairman|chairwoman|chair|senator|secretary|administrator|governor|mayor|ambassador|sir|lord|von|bishop|reverend|rev\.|rabbi|admiral|general|lieutenant)(\s(\w\'|\w{2,}\-)?\w{2,}){1,2}(\s*\[presiding\]\.\s|\s*\[presiding\.\]\s|\sof\s\w{3,}\.\s|\.\s){1}(?!\n)")
regex_chair = "\s{4}the chairman(\s*\[presiding\]\.\s|\s*\[presiding\.\]\s|\.\s){1}(?!\n)"
regex_skip = "\s{4}\[(?=graphic|laughter|laugher|applause|note|slide|chart|see|witness|also|pause|several witnesses|show of hands|chorus of ayes|a moment of silence|nonverbal|audio gap|video|if you have).+"
regex_break = "\s{4}\[(the|recess|brief recess|whereupon|(an )?additional|prepared|submissions|material|information|questions|remarks|editor|end|no response|committee|responses|mr\.|mery|\d\]).+"
regex_break2 = "(\s{4,}|\n\s*)((OPENING )?STATEMENT(S)?|TESTIMONY OF|BIOGRAPHY FOR|______)"

In [10]:
# Parse the hearings
utterances = []
for i, text in enumerate(df.htm):
    hearing_utterances = []
    matched_speaker = []
    utterance = []
    for j, sentence in enumerate(re.split("\n(?=    |\n)", text)):
        # Skip all paragraphs that are not matched to a speaker either by a previous match or by a regex
        if matched_speaker == [] and not re.match(regex_chair, sentence.lower()) and not re.match(regex_speaker, sentence.lower()):
            continue
        # Match the speaker: the chair
        elif re.match(regex_chair, sentence.lower()):
            # print(i, j, re.match(regex_chair, sentence.lower()))
            matched_speaker = re.match(regex_chair, sentence.lower())[0].strip()
            utterance = (j, matched_speaker, sentence[re.match(regex_chair, sentence.lower()).end():].strip())
        # Match the speaker: all other speakers
        elif re.match(regex_speaker, sentence.lower()):
            # print(i, j, re.match(regex_speaker, sentence.lower()))
            matched_speaker = re.match(regex_speaker, sentence.lower())[0].strip()
            utterance = (j, matched_speaker, sentence[re.match(regex_speaker, sentence.lower()).end():].strip())
        # Skip all paragraphs that are matched to a speaker but should be skipped
        elif matched_speaker and re.match(regex_skip, sentence.lower()):
            continue
        # Break the speaker if a breaking regex is found and move to the next sentence
        elif matched_speaker and (re.match(regex_break, sentence.lower()) or re.match(regex_break2, sentence)):
            matched_speaker = []
            continue
        # If a speaker is matched and the sentence is not skipped or broken, append the sentence to the utterance
        elif matched_speaker and not re.match(regex_chair, sentence.lower()) and not re.match(regex_speaker, sentence.lower()) and not re.match(regex_skip, sentence.lower()) and not re.match(regex_break, sentence.lower()) and not re.match(regex_break2, sentence) and not sentence.isupper():
            utterance = (j, matched_speaker, sentence.strip())
        # If a speaker is matched and the sentence is not skipped or broken, but the sentence is all upper case, break the speaker and move to the next sentence
        elif matched_speaker and not re.match(regex_chair, sentence.lower()) and not re.match(regex_speaker, sentence.lower()) and not re.match(regex_skip, sentence.lower()) and not re.match(regex_break, sentence.lower()) and not re.match(regex_break2, sentence) and sentence.isupper():
            matched_speaker = []
            continue
        # Append the utterance to the list of utterances per hearing
        hearing_utterances.append(utterance)
    # Append the list of utterances to the list of all utterances
    utterances.append(hearing_utterances)

df['utterances'] = utterances

In [11]:
df = df.explode(['utterances']).reset_index(drop=True)

In [12]:
df["utterance"] = df.utterances.apply(lambda x: x[0])
df["speaker"] = df.utterances.apply(lambda x: x[1])
df["text"] = df.utterances.apply(lambda x: x[2])
# df

# Match the utterances to the witnesses and MoCs

In [13]:
# Match the witness_name to the speaker

df["witness_match"] = df.apply(lambda x: [i for i in x.witness_name if re.findall(i.lower(), x.speaker)], axis=1)
df["witness_match"] = df.witness_match.apply(lambda x: x[0] if not x == [] else np.nan)


# Match the MoCs to the speaker
df["moc_match"] = df.apply(lambda x: [i for i in x.moc_name if re.findall(i.lower(), x.speaker)], axis=1)
df["moc_match"] = df.apply(lambda x: [x.moc_name[0]] if re.findall("(mr\.|the) chairman", x.speaker) else x.moc_match, axis=1)
df["moc_match"] = df.moc_match.apply(lambda x: x[0] if not x == [] else np.nan)


In [14]:
# Check missing matches, i.e. speakers that can't be matched to a wirtness or committee member
df.loc[df.witness_match.isnull()&df.moc_match.isnull()].iloc[0:200000000000, [0, 35]].drop_duplicates().reset_index(drop=True).head(50)
# packageId       	    speaker               note
# CHRG-109shrg32206	    mr. linton.           (not mentioned who this is)
# CHRG-109shrg32206	    mr. laitos.           (not mentioned who this is)
# CHRG-110hhrg32966	    mr. gilchrest.        Representative Gilchrest (not on committee)
# CHRG-110shrg53827	    senator menendez.     Senator Menendez (not on committee)
# CHRG-110shrg53827	    senator cantwell.     Senator Cantwell (not on committee)
# CHRG-110shrg55923	    senator mikulski.     Senator Mikulski (not on committee)
# CHRG-110hhrg61955	    mr. shays.            Representative Christopher H. Shays, State of Connecticut (not on committee)
# CHRG-110hhrg61955	    mr. murphy.           Representative Christopher Scott Murphy, State of Connecticut (not on committee)
# CHRG-110hhrg61955	    mr. courtney.         Representative Joseph Courtney, State of Connecticut (not on committee)
# CHRG-111shrg47252	    senator tester.       Senator Tester (not on committee)
# CHRG-111hhrg47754	    mr. lovejoy.          Tom Lovejoy, an acclaimed biologist here in Washington
# CHRG-111shrg62715	    mr. thompson.         Representative Mike Thompson, State of California (not on committee)
# CHRG-111shrg99887	    senator kerry.        Senator Kerry (not on committee)
# CHRG-111shrg95164	    senator menendez.     Senator Menendez (not on committee)
# CHRG-111shrg95164	    senator bennet.       Senator Bennet has dropped in to introduce Governor Ritter (not on committee)
# CHRG-111hhrg47755	    mr. syms.             Harold Syms (not mentioned who this is)  >>17

,packageId,speaker
0,CHRG-109shrg32206,mr. linton.
1,CHRG-109shrg32206,mr. laitos.
2,CHRG-110hhrg32966,mr. gilchrest.
3,CHRG-110shrg53827,senator menendez.
4,CHRG-110shrg53827,senator cantwell.
5,CHRG-110shrg55923,senator mikulski.
6,CHRG-110hhrg61955,mr. shays.
7,CHRG-110hhrg61955,mr. murphy.
8,CHRG-110hhrg61955,mr. courtney.
9,CHRG-111shrg47252,senator tester.


In [15]:
# Check duplicate matches, i.e. if the same speaker is matched to a witness and an MoC
df[~df.witness_match.isnull()&~df.moc_match.isnull()]
# Correct duplicate matches
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "109shrg28095") & (x.speaker == "mr. johnson.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "109shrg24631") & (x.speaker == "dr. smith.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "109shrg24631") & (x.speaker == "senator smith.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "109shrg24631") & (x.speaker == "senator smith [presiding].") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "110hhrg35926") & (x.speaker == "mr. johnson.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "110hhrg35926") & (x.speaker == "ms. johnson of texas.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "110hhrg35926") & (x.speaker == "mr. hall.") & (x.utterance <= 526) else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "110hhrg35926") & (x.speaker == "mr. hall.") & (x.utterance >= 583) else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "110hhrg35926") & (x.speaker == "mr. cohen.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "110hhrg36921") & (x.speaker == "mr. inslee.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "110hhrg39491") & (x.speaker == "mr. smith of nebraska.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "110hhrg36921") & (x.speaker == "mr. rogers.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "110shrg34079") & (x.speaker == "dr. smith.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "111hhrg63514") & (x.speaker == "mr. rogers of michigan.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg63825") & (x.speaker == "general sullivan.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg72878") & (x.speaker == "ms. gordon.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "111hhrg72878") & (x.speaker == "mr. gordon.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "111hhrg72878") & (x.speaker == "mr. rogers.") & (x.utterance <= 1169) else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg72878") & (x.speaker == "mr. rogers.") & (x.utterance >= 1416) else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg72878") & (x.speaker == "mr. greenstein.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg67102") & (x.speaker == "mr. greenstein.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg67102") & (x.speaker == "mr. hill.") else np.nan, axis=1)
df["moc_match"] = df.apply(lambda x: x.moc_match if not (x.hearing_id == "111hhrg53787") & (x.speaker == "mr. johnson.") else np.nan, axis=1)
df["witness_match"] = df.apply(lambda x: x.witness_match if not (x.hearing_id == "110shrg73571") & (x.speaker == "senator klobuchar.") else np.nan, axis=1)
# Check if there are any remaining duplicate matches
print(len(df[~df.witness_match.isnull()&~df.moc_match.isnull()]), "more duplicate matches.")

0 more duplicate matches.


In [16]:
# Search for missing witness testimonies
print("Missing testimonies:")
missing = pd.concat([df.hearing_id.drop_duplicates().reset_index(drop=True), 
           df.witness_name.drop_duplicates().reset_index(drop=True),
           df.groupby("hearing_id", sort=False).witness_match.unique().reset_index(drop=True).apply(lambda x: [i for i in x if not pd.isnull(i)])], axis=1)
missing["missing"] = missing.apply(lambda x: [i for i in x.witness_name if i not in x.witness_match], axis=1)
missing[missing.missing.apply(lambda x: len(x) > 0)]

# df.htm[38] # Hearing CHRG-110hhrg56980
# One oral testimony is missing: 
# -- Paul N. Cicio, President, Industrial Energy Consumers of America \\1\\
# "\1\ Mr. Cicio did not present an oral statement at the hearing."

# df.htm[45] # Hearing CHRG-110shrg73571
# -- Klobuchar Amy is MoC

# df.htm[62] # Hearing CHRG-110shrg44664
# One oral testimony is missing: 
# - Hannegan, Bryan, Vice President, Environment and Generation, the Electric Power Research Institute

# df.htm[94] # Hearing CHRG-111shrg20182
# One oral testimony is missing:
# -- Green, Kenneth P., Ph.D., Resident Scholar, American Enterprise Institute
# "our last witness was ill and could not be here, Dr. Green, and we wish him well"

# df.htm[116] #CHRG-111shrg55432
# Two oral testimonies are missing: 
# -- Alterman, Stephen A., President, Cargo Airline Association
# -- Gilligan, Donald, President, National Association of Energy Service Companies           

Missing testimonies:


,hearing_id,witness_name,witness_match,missing
38,110hhrg56980,"[Naasz, Goo, Reuther, Jacobson, Kuhn, Bowman, ...","[Naasz, Goo, Reuther, Jacobson, Kuhn, Bowman, ...",[Cicio]
45,110shrg73571,"[Gabriel, Armey, Khosla, Ringo, Winegarden, Be...","[Gabriel, Armey, Khosla, Ringo, Winegarden, Be...",[Klobuchar]
62,110shrg44664,"[McLean, Hannegan, Orszag, Gruenspecht, Parker...","[Yacobucci, Parker, Gruenspecht, McLean, Orszag]",[Hannegan]
94,111shrg20182,"[Brehm, Reicher, Foster, Nutter, Gordon, Kless...","[Brehm, Reicher, Foster, Nutter, Gordon, Kless...",[Green]
116,111shrg55432,"[Gayer, Alic, Hawkins, Alterman, Kopp, Banks, ...","[Kopp, Gayer, Hawkins, Banks, Alic]","[Alterman, Gilligan]"


# Process the data

In [17]:
df["type"] = np.nan
df.type[~df.moc_match.isnull()] = "MoC"
df.type[~df.witness_match.isnull()] = "witness"

df["last_name"] = np.nan
df.last_name[~df.moc_match.isnull()] = df.moc_match[~df.moc_match.isnull()]
df.last_name[~df.witness_match.isnull()] = df.witness_match[~df.witness_match.isnull()]

df["year"] = df['dateIssued'].apply(lambda x: x[0:4])
df["date"] = df['dateIssued']
df["matched_speaker"] = df['speaker']

In [18]:
# Subset the utterances of the matched witnesses
df_witnesses = df[df.witness_match.notnull()].reset_index(drop=True)

# Select the correct information for the matched witnesses
df_witnesses["witness"] = df_witnesses.apply(lambda x: x.witnesses[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_affiliation"] = df_witnesses.apply(lambda x: x.witness_affiliation[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_category"] = df_witnesses.apply(lambda x: x.category[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_subcategory"] = df_witnesses.apply(lambda x: x.subcategory[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_opensectrets_industry"] = df_witnesses.apply(lambda x: x.witness_industry[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_opensectrets_sector"] = df_witnesses.apply(lambda x: x.witness_sector[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_state_research"] = df_witnesses.apply(lambda x: x.state_research[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_contrarian_organisation"] = df_witnesses.apply(lambda x: x.contrarian_organisation[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_contrarian"] = df_witnesses.apply(lambda x: x.contrarian[x.witness_name.index(x.witness_match)], axis=1)
df_witnesses["witness_denialist"] = df_witnesses.apply(lambda x: x.denialist[x.witness_name.index(x.witness_match)], axis=1)
# Subset the necessary columns
df_witnesses = df_witnesses[['congress', 'chamber', 'committee', 'committee_short', 'year', 'date',
           'title', 'hearing_id', 'type', 'last_name', 'matched_speaker', 'utterance', 'text', 'witness', 'witness_affiliation', 
           'witness_opensectrets_industry', 'witness_opensectrets_sector', 'witness_category', 'witness_subcategory',
           'witness_state_research', 'witness_contrarian_organisation', 'witness_contrarian', 'witness_denialist']]
df_witnesses.iloc[:, 0:13].head()

,congress,chamber,committee,committee_short,year,date,title,hearing_id,type,last_name,matched_speaker,utterance,text
0,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,witness,Colburn,mr. colburn.,158,"Thank you, Mr. Chairman. It's a delight to be ..."
1,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,witness,Colburn,mr. colburn.,159,My name's Ken Colburn. I'm the Executive Direc...
2,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,witness,Colburn,mr. colburn.,160,"The biggest concern I have, Senator, is stayin..."
3,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,witness,Colburn,mr. colburn.,161,"This week is a good example. On Monday, the no..."
4,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,witness,Colburn,mr. colburn.,163,"Senator, I'm not sure if California has those ..."


In [19]:
# Subset the utterances of the matched MoCs
df_mocs = df[df.moc_match.notnull()].reset_index(drop=True)
# Select the correct information for the matched MoCs
df_mocs["MoC_bioguide_id"] = df_mocs.apply(lambda x: x.bioguide_id[x.moc_name.index(x.moc_match)], axis=1)
df_mocs["MoC_govtrack_id"] = df_mocs.apply(lambda x: x.govtrack_id[x.moc_name.index(x.moc_match)], axis=1)
# Subset the necessary columns
df_mocs = df_mocs[['congress', 'chamber', 'committee', 'committee_short', 'year', 'date', 'title', 
                   'hearing_id', 'type', 'last_name',  'matched_speaker', 'utterance', 'text', 'MoC_bioguide_id', 'MoC_govtrack_id']]
# Add the MoC metadata from mocs to the utterance level data
mocs_meta = mocs.loc[:, ['hearing_id', 'congress']].join(mocs.iloc[:, 50:75].add_prefix('MoC_'))
df_mocs = df_mocs.merge(mocs_meta, how='left', on = ['congress', 'hearing_id', 'MoC_bioguide_id', 'MoC_govtrack_id'])
df_mocs.iloc[:, 0:13].head()

,congress,chamber,committee,committee_short,year,date,title,hearing_id,type,last_name,matched_speaker,utterance,text
0,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,MoC,McCain,the chairman.,104,"Good morning. Today, we continue with our \nse..."
1,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,MoC,McCain,the chairman.,105,During the four previous hearings on this issu...
2,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,MoC,McCain,the chairman.,106,"Arizona, my home state, is facing record droug..."
3,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,MoC,McCain,the chairman.,107,While I'm concerned about what's occurring in ...
4,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2004,2004-05-06,The Impacts of Climate Change and States' Actions,108shrg82493,MoC,McCain,the chairman.,108,We will also hear from those who must deal wit...


In [20]:
# Subset the unmatched utterances
df_unmatched = df[df.witness_match.isnull() & df.moc_match.isnull()].reset_index(drop=True)
# Subset the necessary columns
df_unmatched = df_unmatched[['congress', 'chamber', 'committee', 'committee_short', 'year', 'date', 'title', 'hearing_id', 'type', 'last_name', 'matched_speaker', 'utterance', 'text']]
df_unmatched.iloc[:, 0:13].head()

,congress,chamber,committee,committee_short,year,date,title,hearing_id,type,last_name,matched_speaker,utterance,text
0,109,SENATE,Committee on Environment and Public Works,Environment and Public Works,2005,2005-02-02,"S. 131, ``the Clear Skies Act of 2005''",109shrg32206,NaN,NaN,mr. linton.,1841,I'm just going to ask if Jan has any comments ...
1,109,SENATE,Committee on Environment and Public Works,Environment and Public Works,2005,2005-02-02,"S. 131, ``the Clear Skies Act of 2005''",109shrg32206,NaN,NaN,mr. laitos.,1842,What do you think the chances are within the n...
2,109,SENATE,Committee on Environment and Public Works,Environment and Public Works,2005,2005-02-02,"S. 131, ``the Clear Skies Act of 2005''",109shrg32206,NaN,NaN,mr. laitos.,1846,"One more question. Do you think, based on your..."
3,109,SENATE,Committee on Environment and Public Works,Environment and Public Works,2005,2005-02-02,"S. 131, ``the Clear Skies Act of 2005''",109shrg32206,NaN,NaN,mr. linton.,1851,Thank you.
4,109,SENATE,Committee on Environment and Public Works,Environment and Public Works,2005,2005-02-02,"S. 131, ``the Clear Skies Act of 2005''",109shrg32206,NaN,NaN,mr. linton.,1852,"Statement of Abraham Breehey, Legislative Repr..."


In [21]:
# Rejoin the dataframes
df_all_rows = pd.concat([df_witnesses, df_mocs, df_unmatched], axis=0, ignore_index=True)
df_all_rows.sort_values(by=['date', 'hearing_id', 'utterance'], inplace=True)
df_all_rows.reset_index(drop=True, inplace=True)
df_all_rows.head()

,congress,chamber,committee,committee_short,year,date,title,hearing_id,type,last_name,...,MoC_state_icpsr,MoC_cd_code,MoC_stab,MoC_notes,MoC_Party,MoC_CID,MoC_last_name,MoC_first_name,MoC_birthday,MoC_gender
0,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2003,2003-01-08,Climate Change,108shrg95341,MoC,McCain,...,61.0,83.0,AZ,NaN,R,N00006424,McCain,John,1936-08-29,M
1,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2003,2003-01-08,Climate Change,108shrg95341,MoC,McCain,...,61.0,83.0,AZ,NaN,R,N00006424,McCain,John,1936-08-29,M
2,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2003,2003-01-08,Climate Change,108shrg95341,MoC,McCain,...,61.0,83.0,AZ,NaN,R,N00006424,McCain,John,1936-08-29,M
3,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2003,2003-01-08,Climate Change,108shrg95341,MoC,McCain,...,61.0,83.0,AZ,NaN,R,N00006424,McCain,John,1936-08-29,M
4,108,SENATE,"Committee on Commerce, Science, and Transporta...","Commerce, Science, and Transportation",2003,2003-01-08,Climate Change,108shrg95341,MoC,McCain,...,61.0,83.0,AZ,NaN,R,N00006424,McCain,John,1936-08-29,M


# Corrections

In [22]:
# Fix CO<INF>2</INF> and other similar cases 
df_all_rows["text"] = df_all_rows.text.str.replace("<INF>", "").str.replace("</INF>", "").str.replace("O\\\\2\\\\", "O2")
# Remove titles inbetween utterances
df_all_rows["text"] = df_all_rows.text.apply(lambda x: re.sub("^[A-Z]{3,} [A-Z]{2,} ([A-Z]{2,} )*", "", x))
# Remove newlines
df_all_rows["text"] = df_all_rows.text.apply(lambda x: re.sub(" \n", " ", x))
# Remove multiple spaces
df_all_rows["text"] = df_all_rows.text.apply(lambda x: re.sub(" +", " ", x))
# Remove leading and trailing spaces
df_all_rows["text"] = df_all_rows.text.str.strip()
# Drop empty rows
df_all_rows = df_all_rows[df_all_rows.text != ""]
df_all_rows = df_all_rows.reset_index(drop=True)

In [23]:
# Sort the data in reverse to iterate over the data backwards in order to be able to join multiple rows that were accidentally split
df_all_rows.sort_values(by=['date', 'hearing_id', 'utterance'], inplace=True, ascending=False)
df_all_rows.reset_index(drop=True, inplace=True)
# Join data rows that are wrongly split by joining the rows that start with a lower case letter that have the same last name as the utterance before (here +1 due to the reversed order) to that utterance
counter = 0
for i, t in enumerate(df_all_rows.text):
    if re.match("[a-z]", t) and df_all_rows.last_name[i+1] == df_all_rows.last_name[i]:
        # print(i)
        # print(df_all_rows.text[i+1])
        # print(df_all_rows.text[i])
        # print(df_all_rows.last_name[i+1], df_all_rows.last_name[i])
        df_all_rows.text[i+1] = df_all_rows.text[i+1] + " " + df_all_rows.text[i]
        df_all_rows.text[i] = np.nan
        counter += 1
print(counter, "rows were joined, resulting in", len(df_all_rows) - counter, "rows instead of", len(df_all_rows), "rows.")
# Drop the empty rows and reset the order
df_all_rows.dropna(subset=['text'], inplace=True)
df_all_rows.sort_values(by=['date', 'hearing_id', 'utterance'], inplace=True, ascending=True)
df_all_rows.reset_index(drop=True, inplace=True)

4889 rows were joined, resulting in 68793 rows instead of 73682 rows.


In [24]:
# Join data rows that are wrongly split by joining the rows that end with a word (i.e. no punctuation) and that have the same last name as the following utterance to that utterance for all rows in hearing 109hhrg31362
df_all_rows.sort_values(by=['date', 'hearing_id', 'utterance'], inplace=True, ascending=True)
df_all_rows.reset_index(drop=True, inplace=True)
# Join data rows that are wrongly split by joining the rows that start with a lower case letter that have the same last name as the utterance before (here +1 due to the reversed order) to that utterance
counter = 0
for i, t in enumerate(df_all_rows.text):
    if df_all_rows.hearing_id[i] == "109hhrg31362" and re.findall("([a-z]|,)$", t) and df_all_rows.last_name[i+1] == df_all_rows.last_name[i]:
        # print(i, df_all_rows.hearing_id[i])
        # print(df_all_rows.text[i])
        # print(df_all_rows.text[i+1])
        # print(df_all_rows.last_name[i+1], df_all_rows.last_name[i])
        df_all_rows.text[i+1] = df_all_rows.text[i] + " " + df_all_rows.text[i+1]
        df_all_rows.text[i] = np.nan
        counter += 1
print(counter, "rows were joined, resulting in", len(df_all_rows) - counter, "rows instead of", len(df_all_rows), "rows.")
# Drop the empty rows
df_all_rows.dropna(subset=['text'], inplace=True)
df_all_rows.reset_index(drop=True, inplace=True)

417 rows were joined, resulting in 68376 rows instead of 68793 rows.


In [25]:
# Manually join data rows that are split differently in the first version of the parsing algorithm in order to match the labelled data to the newly parsed data
regex_corrections_1 = "Well, presumably there is some element of that|I would say it has been an uphill fight|Well, a couple of points|We did not attempt to reproduce|Figure 5 shows|Last week I pointed out|Dr. Mann likes to say|Well, that is actually|Well, because you have heard|I cannot say that|Well, I can understand that|Finally, at the NAS press conference|In the two reports|I have three main messages|Figure 4 is an image|The NRC panel illustrated|If you would submit"
regex_corrections_2 = "I think one should take|The Wegman reported noted|When asked by others"
regex_corrections_3 = "Some comments of Dr."
counter = 0
for i, t in enumerate(df_all_rows.text):
    if df_all_rows.hearing_id[i] == "109hhrg31362" and re.findall(regex_corrections_1, t) and df_all_rows.last_name[i+1] == df_all_rows.last_name[i]:
        # # print(i, df_all_rows.hearing_id[i])
        # # print(df_all_rows.text[i])
        # # print(df_all_rows.text[i+1])
        df_all_rows.text[i] = df_all_rows.text[i] + " " + df_all_rows.text[i+1]
        df_all_rows.text[i+1] = "TO_DROP"
        counter += 1
    if df_all_rows.hearing_id[i] == "109hhrg31362" and re.findall(regex_corrections_2, t) and df_all_rows.last_name[i+2] == df_all_rows.last_name[i+1] == df_all_rows.last_name[i]:
        # print(i, df_all_rows.hearing_id[i])
        # print(df_all_rows.text[i])
        # print(df_all_rows.text[i+1])
        # print(df_all_rows.text[i+2], "\n")
        df_all_rows.text[i] = df_all_rows.text[i] + " " + df_all_rows.text[i+1] + " " + df_all_rows.text[i+2]
        df_all_rows.text[i+1] = "TO_DROP"
        df_all_rows.text[i+2] = "TO_DROP"
        counter += 2
    if df_all_rows.hearing_id[i] == "109hhrg31362" and re.findall(regex_corrections_3, t) and df_all_rows.last_name[i+3] == df_all_rows.last_name[i+2] == df_all_rows.last_name[i+1] == df_all_rows.last_name[i]:
        # print(i, df_all_rows.hearing_id[i])
        # print(df_all_rows.text[i])
        # print(df_all_rows.text[i+1])
        # print(df_all_rows.text[i+2])
        # print(df_all_rows.text[i+3], "\n")
        df_all_rows.text[i] = df_all_rows.text[i] + " " + df_all_rows.text[i+1] + " " + df_all_rows.text[i+2] + " " + df_all_rows.text[i+3]
        df_all_rows.text[i+1] = "TO_DROP"
        df_all_rows.text[i+2] = "TO_DROP"
        df_all_rows.text[i+3] = "TO_DROP"
        counter += 3
print(counter, "rows were joined, resulting in", len(df_all_rows) - counter, "rows instead of", len(df_all_rows), "rows.")
# Drop the empty rows
df_all_rows = df_all_rows[~(df_all_rows.text == "TO_DROP")]
df_all_rows.dropna(subset=['text'], inplace=True)
df_all_rows.reset_index(drop=True, inplace=True)

26 rows were joined, resulting in 68350 rows instead of 68376 rows.


In [26]:
# Check long statements
for i, x in enumerate(df_all_rows.text):
    if len(x.split())>400:
        print(i, x, "\n")

7765 Now, the United Nations Intergovernmental Panel on Climate Change is the world body with most of the interest and does focus on this subject of global warming. And it is the body that most people look to on this subject. Now, for many years the Intergovernmental Panel on Climate Change used a chart that clearly shows the temperature from 1000 A.D. to about 1450 A.D., that the temperatures during that period were significantly warmer than the latter part of the 20th Century, or the late 1990s. Now, in 1998 and 1999, a
 paleoclimatologist, Dr. Michael Mann, with Raymond Bradley and Dr. Malcolm Hughes, introduced a new technique to develop more quantitative estimates of the nature of climate change since 1000 A.D. and concluded that the late 20th Century was the warmest in 1,000 years, that the warming during the late 1990s was the warmest in over 1,000 years. Now, as a result of that report, the IPCC incorporated the study with other data which eliminated the warming period for 1000

# Export the data

In [27]:
# Save the hearings with parsed utterances
df_all_rows.to_json("../Data/cat_hearings_03_10_utterances_witnesses_MoCs.json")